# Phase 2 TMDB ETL

# Business Problem

## After investigating the preview of the data from the IMDB ETL, the stakeholder realized that there is no financial information included in the IMDB data (e.g. budget or revenue).

## This will be a major roadblock when attempting to analyze which movies are successful and must be addressed before you will be able to determine which movies are successful.

## Specifications
- The stakeholder would like the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification", extracted.

- The stakeholder would only like results for movies that meet all of the criteria established in the IMDB ETL extracted.

- As a proof-of-concept, they requested a test extraction of movies that started in 2000 or 2001. Each year to be saved as a separate .csv.gz file.

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [2]:
import json
with open('/Users/parri_nqdmzn3/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [3]:
# Set the API_KEY variable to the "API Key(v3 auth)"
tmdb.API_KEY =  login['api-key']

## Test API call

In [4]:
# Create a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

In [5]:
# Display the .info dictionary of the movie object
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/oMsxZEvz9a708d49b6UdZK1KAo5.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 76.366,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

In [6]:
info['budget']

63000000

In [7]:
info['revenue']

463517383

In [8]:
info['imdb_id']

'tt0133093'

In [9]:
# Test search
movie = tmdb.Movies('tt0848228')
info = movie.info()
info['budget']

220000000

In [10]:
# Test search
movie = tmdb.Movies('tt0332280')
info = movie.info()
info['budget']

29000000

## API Call

In [11]:
# Create a data folder to save API call data if it doesn't exist
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_basics.csv.gz', 'tmdb_api_results_2000.json']

In [12]:
YEARS_TO_GET = [2000,2001]

In [13]:
# Function to obtain movie rating
def get_movie_with_rating(movie_id):
    # Get movie and release dates
    movie = tmdb.Movies(movie_id)
    # Construct output dictionary
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            movie_info['certification'] = c['certification']
    return movie_info

## JSON

In [14]:
# Append new results to the existing JSON file
# Adapted from: 
# https://www.geeksforgeeks.org/append-to-json-file-using-python/   

def write_json(new_data, filename): 
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [17]:
# Load in the IMDB title_basics file
basics = pd.read_csv("Data/title_basics.csv.gz")

In [18]:
# OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    # Define the JSON file to store results for current Year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
    # Save an empty dict with just "imdb_id" to the new JSON file
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
               
    # Filter the IMDB title_basics file on current Year
    df = basics.loc[ basics['startYear']==YEAR].copy()
    
    # Save IMDB title_basics file movie ids to list
    movie_ids = df['tconst'].copy()
    # Create a dataframe from the JSON file
    previous_df = pd.read_json(JSON_FILE)
    
    # Filter out movie ids that are already exist in the JSON file
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

# INNER Loop
    # Get index and current movie id from list
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve the data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)
            # Append/extend results to JSON file function
            write_json(temp,JSON_FILE)
            # Pause 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails, make a dict with just the id and None for certification.
        except Exception as e:
            continue
            
    # Save the year's results as csv.gz file
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", 
                         compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/399 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1573 [00:00<?, ?it/s]